<a href="https://colab.research.google.com/github/SaehyunC/localization/blob/master/answer/localization_assignment3_1D_Particle_Filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# sim.plot

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import gridspec
import numpy as np

distance = 40


def create_poles(poles):
    y = np.zeros(distance)
    for p in poles:
        y[p] = 1
    x = range(distance)
    plt.stem(x, y, use_line_collection=True)


def plot_robot_measurement(poles, pos, gs):
    plt.subplot(gs[2:3, 0])
    plt.yticks([])
    plt.xticks([])
    plt.xlim([pos - 1.5, pos + 3.5])
    plt.ylim([-0.1, 1.1])
    plt.plot([pos + 0.2], [0.6], 'g<', markersize=40)
    plt.plot([pos], [0.4], 'bo', markersize=40)
    create_poles(poles)


def plot_simple(particles, poles, pos=None, j=None):
    gs = gridspec.GridSpec(3, 1)
    # Plot Main Display
    plt.subplot(gs[0:2, 0])
    if j is not None:
        plt.title(str(j))
    plt.yticks([])
    plt.xlim([-0.9, distance + 0.9])
    for particle in particles:
        if particle.belief == 0:
            continue
        plt.plot([particle.pos], [0.5], '*', color=particle.color)
    create_poles(poles)

    # Plot Robot Measurement
    if pos is not None:
        plot_robot_measurement(poles, pos, gs)
    plt.show(block=True)


def plot(
        particles,
        poles,
        pos,
        resampled_particles=None,
        j=None,
        autorun=False):
    gs = gridspec.GridSpec(3, 1)
    # Plot Main Display
    plt.subplot(gs[0:2, 0])
    if j is not None:
        plt.title(str(j))
    plt.yticks([])
    plt.xlim([-0.9, distance + 0.9])
    for particle in particles:
        plt.plot([particle.pos], [0.5], 'b*', label="Particles")
    if resampled_particles is not None:
        for particle in resampled_particles:
            plt.plot([particle.pos], [0.25], 'g*', label="Resampled")
    plt.plot([pos], [0.65], 'r*', label="Robot")
    # Remove duplicates in legend (because of way I plotted one at a time.
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys(), loc='upper right')

    create_poles(poles)

    # Plot Robot Measurement
    if pos is not None:
        plot_robot_measurement(poles, pos, gs)
    if autorun:
        if j == 0:
            # Not sure why this is needed but it is.
            plt.pause(1)
        plt.show(block=False)
        plt.pause(1)
        plt.close()
    else:
        plt.show()

def print_particle_error(robot, particles):
    weights = []
    for particle in particles:
        weights += [particle.weight]
    best_particle = weights.index(max(weights))
    print("Error: " +
          str(round(abs(particles[best_particle].pos - robot.pos), 2)))
    print("Weight Sum: " + str(sum(weights)))
    print()

# assignment 3-1

In [ ]:
#from sim.plot import plot_simple
import random as r


class Robot:
    def __init__(self, pos):
        self.pos = pos
        self.pole_detected = False
        self.move_dist = 1

    def move(self):
        self.pos += self.move_dist

    def detect_pole(self, poles):
        if self.pos + 1 in poles:
            self.pole_detected = True
        else:
            self.pole_detected = False


class Particle(Robot):
    def __init__(self, pos, color):
        # If you don't understand this, see derived_classes.py in my Python
        # Intro Course mentioned in the 2nd lecture.
        Robot.__init__(self, pos)
        self.belief = 1
        self.color = color

    def predict(self):
        # START STUDENT CODE
        # Move the particle the same distance as the robot moves.

        # END STUDENT CODE

    def update_belief(self, robot_pole_detected):
        # START STUDENT CODE
        # Set the belief to 0 if the robot detection and the particle detection
        # don't match.

        # END STUDENT CODE

        # Setup Robot Location
robot = Robot(5)
poles = [10, 15]

# Setup particles.
num_of_locations = 40
particle_spacing = 1
print("Starting Number of Particles: " +
      str(int(num_of_locations / particle_spacing)))
particles = []
for i in range(0, num_of_locations, particle_spacing):
    color = (r.random(), r.random(), r.random(), 1)
    particles += [Particle(i, color)]

# Plot starting distribution, no beliefs
plot_simple(particles, poles)

# Begin Calculating
for j in range(12):
    # Move
    if j != 0:
        robot.move()
        for particle in particles:
            particle.predict()

    # Measure
    robot.detect_pole(poles)
    for particle in particles:
        particle.detect_pole(poles)

        # Update Beliefs
        particle.update_belief(robot.pole_detected)

    plot_simple(particles, poles, robot.pos, j)

# assignment 3-2

In [ ]:
import matplotlib.pyplot as plt
import math
import numpy as np


class Robot:
    def __init__(self, pos):
        self.pos = pos
        self.move_dist = 1

    # Movement is perfectly accurate, even though we are assuming it isn't.
    def move(self):
        self.pos += self.move_dist


class Particle(Robot):
    def __init__(self, pos):
        Robot.__init__(self, pos)
        self.movement_sigma = 0.2

    def predict(self):
        # START STUDENT CODE
        # Predict the robots movement and account for movement uncertainty.

        # END STUDENT CODE


particle = Particle(0)
# Move 10 times.
print("Particle Pos: " + str(particle.pos))
for i in range(10):
    particle.predict()
    print("Particle Pos: " + str(particle.pos))

# Remove quit() to see how distribution converges with more samples.
# quit()

sample_count = 1
for i in range(8):
    x = np.random.normal(
        particle.move_dist,
        particle.movement_sigma,
        sample_count)

    # the histogram of the data
    n, bins, patches = plt.hist(x, 50, facecolor='green', alpha=0.75)
    plt.grid(True)
    plt.xlim([0, 2])
    print(sample_count)
    plt.show(block=False)
    plt.pause(2)
    if i == 7:
        plt.pause(100)
    plt.close()

    sample_count *= 10

# assignment 3-3

In [ ]:
import matplotlib.pyplot as plt
import math
import numpy as np
import random as r


class Robot:
    def __init__(self, pos):
        self.pos = pos
        self.pole_dist = -100
        self.max_measurement = 3

    # Measurement is perfectly accurate even though we are assuming it isn't.
    def measure(self, poles):
        ### START STUDENT CODE
        # Set self.pole_dist to the distance to the closest pole.

        ### END STUDENT CODE

class Particle(Robot):
    def __init__(self, pos):
        Robot.__init__(self, pos)



poles = [1, 10]
particle = Particle(0.1)
particle.measure(poles)
print("Output Should Be 0.9")
print("You Calculated: " + str(round(particle.pole_dist, 1)))
print()

particle.pos = 11
particle.measure(poles)
print("Output Should Be -100")
print("You Calculated: " + str(round(particle.pole_dist, 1)))
print()

particle.pos = 6.9
particle.measure(poles)
print("Output Should Be -100")
print("You Calculated: " + str(round(particle.pole_dist, 1)))
print()

particle.pos = 7.1
particle.measure(poles)
print("Output Should Be 2.9")
print("You Calculated: " + str(round(particle.pole_dist, 1)))
print()

particle.pos = 9.5
particle.measure(poles)
print("Output Should Be 0.5")
print("You Calculated: " + str(round(particle.pole_dist, 1)))
print()

# assignment 3-4

In [ ]:
import matplotlib.pyplot as plt
import math
import numpy as np
import random as r


class Robot:
    def __init__(self, pos):
        self.pos = pos
        self.pole_dist = 0


class Particle(Robot):
    def __init__(self, pos):
        Robot.__init__(self, pos)
        self.weight = 0
        self.measurement_sigma = 0.5

    def probability_density_function(self, mu, x):
        ### STUDENT CODE START
        ### STUDENT CODE END

    def update_weight(self, robot_dist):
        ### STUDENT CODE START
        ### STUDENT CODE START


# Plot Weights for a range of robot measurements.
particle = Particle(0.0)
x = np.arange(-5, 5, 0.01)
y = np.zeros(len(x))
for i in range(len(x)):
    particle.update_weight(x[i])
    y[i] = particle.probability_density_function(0, x[i])

plt.plot(x, y, '-r')
plt.grid(True)
plt.show()

# Integrate left side to calculate probablity.
sum_probability = 0
for i in range(int(len(y) / 2)):
    sum_probability += y[i]

print("If Probability is close to 0.5, then PDF works.")
print(round(sum_probability * 0.01, 2))
print()

# Update Particle Weigth based on robot measurement.
robot_dist = 3.0
particle.pole_dist = 3.0
particle.update_weight(robot_dist)
print("Particle Weight: " + str(round(particle.weight, 2)))
plt.plot(x, y, '-r')
plt.plot([-5, 5], [particle.weight, particle.weight], '-b')
plt.grid(True)
plt.show()

# sim.a35

In [ ]:
import matplotlib.pyplot as plt
import math


def plot_particles(particles, distance, show=True):
    plt.xlim([-0.9, distance + 0.9])
    for particle in particles:
        plt.plot([particle.pos], [0.0], '*', color=particle.color)
    if show:
        plt.show()


def plot_resample_counts(particles, resample, i_count, distance, show=True):
    plot_particles(particles, distance, show=False)
    for i in range(len(particles)):
        i_count += [resample.count(i)]
        plt.plot([particles[i].pos, particles[i].pos],
                 [0.0, -i_count[-1]], 'g-')
    if show:
        plt.show()


def plot_resampled(
        particles,
        resample,
        i_count,
        resampled_particles,
        distance,
        show=True,
        move=False):
    plot_particles(particles, distance, show=False)
    plot_resample_counts(particles, resample, i_count, distance, show=False)
    for particle in resampled_particles:
        if move:
            particle.predict()
        plt.plot([particle.pos], [-max(i_count)], '*', color=particle.color)
    if show:
        plt.show()


def plot(particles, resampled_particles, resample, distance):
    i_count = []
    # Plot 1
    plot_particles(particles, distance)
    # Plot 2
    plot_resample_counts(particles, resample, i_count, distance)
    # Plot 3
    plot_resampled(particles, resample, i_count, resampled_particles, distance)
    # Plot 4
    plot_resampled(
        particles,
        resample,
        i_count,
        resampled_particles,
        distance,
        move=True)

# assignment 3-5

In [ ]:
#from sim.a35 import plot

import random as r
import numpy as np


class Robot:
    def __init__(self, pos):
        self.pos = pos
        self.move_dist = 1


class Particle(Robot):
    def __init__(self, pos):
        Robot.__init__(self, pos)
        self.weight = 0
        self.movement_sigma = 1
        self.move_dist = 4  # Overwrite move distance for this example
        self.color = (0, 0, 1, 1)

    def predict(self):
        ### START STUDENT CODE
        ### END STUDENT CODE

def resample_particles(particles):
    ### START STUDENT CODE
    # Please fill this array with the output of the r.choices function.
    resample = []
    print(resample)

    # Please fill this array with resampled partciles.
    resampled_particles = []

    ### END STUDENT CODE

    # Set all resampled particles to a different color.
    for i in resample:
        resampled_particles[-1].color = (0, 1, 0, 1)

    # Plot and return resampled particles.
    plot(particles, resampled_particles, resample, distance)
    return resampled_particles


num_particles = 10
distance = 40
particles = []

# Set first few particles to different weight and color.
for i in range(num_particles):
    particles += [Particle(r.uniform(0, distance))]
    if i < 3:
        particles[-1].weight = 1
        particles[-1].color = (1, 0, 0, 1)
    else:
        particles[-1].weight = 0.1

# Resample Particles
resampled_particles = resample_particles(particles)

# sim.plot

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import gridspec
import numpy as np

distance = 40


def create_poles(poles):
    y = np.zeros(distance)
    for p in poles:
        y[p] = 1
    x = range(distance)
    plt.stem(x, y, use_line_collection=True)


def plot_robot_measurement(poles, pos, gs):
    plt.subplot(gs[2:3, 0])
    plt.yticks([])
    plt.xticks([])
    plt.xlim([pos - 1.5, pos + 3.5])
    plt.ylim([-0.1, 1.1])
    plt.plot([pos + 0.2], [0.6], 'g<', markersize=40)
    plt.plot([pos], [0.4], 'bo', markersize=40)
    create_poles(poles)


def plot_simple(particles, poles, pos=None, j=None):
    gs = gridspec.GridSpec(3, 1)
    # Plot Main Display
    plt.subplot(gs[0:2, 0])
    if j is not None:
        plt.title(str(j))
    plt.yticks([])
    plt.xlim([-0.9, distance + 0.9])
    for particle in particles:
        if particle.belief == 0:
            continue
        plt.plot([particle.pos], [0.5], '*', color=particle.color)
    create_poles(poles)

    # Plot Robot Measurement
    if pos is not None:
        plot_robot_measurement(poles, pos, gs)
    plt.show(block=True)


def plot(
        particles,
        poles,
        pos,
        resampled_particles=None,
        j=None,
        autorun=False):
    gs = gridspec.GridSpec(3, 1)
    # Plot Main Display
    plt.subplot(gs[0:2, 0])
    if j is not None:
        plt.title(str(j))
    plt.yticks([])
    plt.xlim([-0.9, distance + 0.9])
    for particle in particles:
        plt.plot([particle.pos], [0.5], 'b*', label="Particles")
    if resampled_particles is not None:
        for particle in resampled_particles:
            plt.plot([particle.pos], [0.25], 'g*', label="Resampled")
    plt.plot([pos], [0.65], 'r*', label="Robot")
    # Remove duplicates in legend (because of way I plotted one at a time.
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys(), loc='upper right')

    create_poles(poles)

    # Plot Robot Measurement
    if pos is not None:
        plot_robot_measurement(poles, pos, gs)
    if autorun:
        if j == 0:
            # Not sure why this is needed but it is.
            plt.pause(1)
        plt.show(block=False)
        plt.pause(1)
        plt.close()
    else:
        plt.show()

def print_particle_error(robot, particles):
    weights = []
    for particle in particles:
        weights += [particle.weight]
    best_particle = weights.index(max(weights))
    print("Error: " +
          str(round(abs(particles[best_particle].pos - robot.pos), 2)))
    print("Weight Sum: " + str(sum(weights)))
    print()

# assignment 3-6

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random as r
import math
#from sim.plot import plot, print_particle_error


AUTORUN = False
robot_start = 7
num_particles = 20
distance = 40
poles = [10, 15, 17, 19, 30, 39]


### START STUDENT CODE
class Robot:
    def __init__(self, pos):

    # Movement is perfectly accurate, even though we are assuming it isn't.
    def move(self):

    # Measurement is perfectly accurate even though we are assuming it isn't.
    def measure(self, poles):


class Particle(Robot):
    def __init__(self, pos):

    def predict(self):

    def probability_density_function(self, mu, x):

    def update_weight(self, robot_dist):


def resample_particles(particles):
    # Potentially resample uniformly if weights are so low.
    return resampled_particles



def initialize_particles(particles):


### END STUDENT CODE

robot = Robot(robot_start)

# Setup particles.
particles = []
initialize_particles(particles)

# Plot starting distribution, no beliefs
plot(particles, poles, robot.pos)

# Begin Calculating
for j in range(39 - robot.pos):
    # Move
    if j != 0:
        robot.move()
        for particle in particles:
            particle.predict()

    # Measure
    robot.measure(poles)
    for particle in particles:
        particle.measure(poles)

        # Update Beliefs
        particle.update_weight(robot.pole_dist)

    print_particle_error(robot, particles)

    # Resample
    resampled_particles = resample_particles(particles)
    plot(particles, poles, robot.pos, resampled_particles, j, AUTORUN)
    particles = resampled_particles

plot(particles, poles, robot.pos, resampled_particles)